In [1]:
import os
import statistics
import nengo
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import nengo_dl
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, ShuffleSplit
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense
from keras import backend as K

In [2]:
# global config
dataset_path = os.path.join('..', 'datasets', 'VarekaGTNEpochs.mat')
os.makedirs('nengo', exist_ok=True)

# set seed for consistent result
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
def get_dataset(file = dataset_path):
    """
    Helper function to get dataset from file
    """

    np_file = loadmat(file) # load dataset file with matrices
    target_data, non_target_data = np_file['allTargetData'], np_file['allNonTargetData'] # get target and non-target data
    features = np.concatenate((target_data, non_target_data)) # concatenate target and non-target into features

    # target labels are represented as (1, 0) vector, non target labels are represented as (0, 1) vector
    target_labels = np.tile(np.array([1, 0]), (target_data.shape[0], 1)) # set 'target' as (1, 0) vector
    non_target_labels = np.tile(np.array([0, 1]), (non_target_data.shape[0], 1)) # set 'non target' as (0, 1) vector
    labels = np.vstack((target_labels, non_target_labels)) # concatenate target and non target labels

    # filter noise above 100 mV
    threshold = 100.0
    x_result, y_result = [], []
    for i in range(features.shape[0]):
        if not np.max(np.abs(features[i])) > threshold:
            x_result.append(features[i])
            y_result.append(labels[i])

    features, labels = np.array(x_result), np.array(y_result)
    features = features.reshape((features.shape[0], 1, -1))
    labels = labels.reshape((labels.shape[0], 1, -1))

    print('features:', features, features.shape)
    print('labels:', labels, labels.shape)
    return features, labels

In [4]:
def create_model():
    """
    Function to create tensorflow model
    """
    inp = Input(shape=(3, 1200, 1), name='input_layer')
    conv2d = Conv2D(filters=6, kernel_size=(3, 3), activation=tf.nn.relu)(inp)
    dropout1 = Dropout(0.5, seed=0)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(1, 8), padding='same')(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation=tf.nn.relu)(flatten)
    batch_norm = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5, seed=0)(batch_norm)
    output = Dense(2, activation=tf.nn.softmax, name='output_layer')(dropout2)

    return Model(inputs=inp, outputs=output)

In [5]:
def run_ann(model, train, valid, test, params_save_path, iteration, shuffle_training=True):
    """
    Run ann via Nengo simulator. This fits given model with training data (train) and validates it using validation
    data (valid). Then accuracy is calculated using test data (test) and weights are saved to params_save_path
    :param shuffle_training: whether to shuffle data
    :param model: tensorflow model created from create_model() function
    :param train: pair of features and labels from training data
    :param valid: pair of features and labels from validation data
    :param test: pair of features and labels from test data
    :param params_save_path: output path to save weights of the network for SNN testing
    :return accuracy on test data
    """
    x_train, y_train = train[0], train[1]
    x_valid, y_valid = valid[0], valid[1],
    x_test, y_test = test[0], test[1]

    converter = nengo_dl.Converter(model)
    with nengo_dl.Simulator(converter.net, minibatch_size=16) as simulator:
        # some data will get truncated due to batch size
        simulator.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )

        input_layer = converter.inputs[model.get_layer('input_layer')] # get nengo input
        output_layer = converter.outputs[model.get_layer('output_layer')] # get nengo output

        simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            validation_data=({ input_layer: x_valid }, { output_layer: y_valid }),
            epochs=30,
            shuffle=shuffle_training,
            callbacks=[EarlyStopping(patience=5, verbose=1, restore_best_weights=True)] # early stop to avoid overfitting
        ) # train model

        simulator.save_params(params_save_path) # save params for SNN
        ann_eval = simulator.evaluate(x={ input_layer: x_test }, y={ output_layer: y_test }) # evaluate accuracy
        print('{}. ann accuracy: {:5f}%'.format(iteration, ann_eval['probe_accuracy'] * 100)) # log accuracy
        return ann_eval['probe_accuracy'] # return accuracy

def run_snn(model, test, params_load_path, timesteps, scale_firing_rates, synapse, iteration):
    """

    :param model:
    :param test:
    :param params_load_path:
    :param timesteps:
    :param scale_firing_rates:
    :param synapse:
    :param iteration:
    :return:
    """
    converter = nengo_dl.Converter(
        model=model,
        swap_activations={ tf.nn.relu: nengo.SpikingRectifiedLinear() },
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    )

    x_test, y_test = test[0], test[1]

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    input_layer = converter.inputs[model.get_layer('input_layer')]
    output_layer = converter.outputs[model.get_layer('output_layer')]

    x_test_time_tiled = np.tile(x_test, (1, timesteps, 1)) # tile x_test to match desired timesteps for simulator

    with nengo_dl.Simulator(converter.net, minibatch_size=41, progress_bar=False) as simulator:
        simulator.load_params(params_load_path)

        predictions = simulator.predict({ input_layer: x_test_time_tiled })[output_layer] # get results from prediction
        predictions = predictions[:,-1,:] # get last timestep

        predictions = np.argmax(predictions, axis=-1) # get argmax
        y_test = np.squeeze(y_test, axis=1) # remove time dimension from labels since its not relevant
        y_test = np.argmax(y_test, axis=-1) # get argmax of y test as well for comparison

        snn_avg = (predictions == y_test).mean()

        if synapse is None:
            print('{}. snn [timesteps={}, scale_firing_rates={}, synapse=None], accuracy: {:4f}%'
                .format(iteration, timesteps, scale_firing_rates, snn_avg*100))
        else:
            print('{}. snn [timesteps={}, scale_firing_rates={}, synapse={:5f}], accuracy: {:4f}%'
                .format(iteration, timesteps, scale_firing_rates, synapse, snn_avg*100))

        return snn_avg

In [6]:
features, labels = get_dataset()
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.25, random_state=seed, shuffle=True)

print('train_x: {}, shape: {}, train_y: {}, shape: {}'.format(x_train, x_train.shape, y_train, y_train.shape))
print('test_x: {}, shape: {}, test_y: {}, shape: {}'.format(x_test, x_test.shape, y_test, y_test.shape))

ann, snn = [], []

# hyper parameters with format: timesteps, scale_firing_rates, synapse
snn_hyper_params = [
    [30, 1000, 0.01],
    [10, 1, None],
    [30, 500, 0.01],
    [30, 1, 0.01],
    [30, 500, 0.1]
]

iteration = 1
monte_carlo = ShuffleSplit(n_splits=30, test_size=.25, random_state=seed)
for train, valid in monte_carlo.split(x_train):
    print('Iteration: {}'.format(iteration)) # No. iteration

    x_train_current, y_train_current = x_train[train], y_train[train]
    x_valid_current, y_valid_current = x_train[valid], y_train[valid]

    params_path = os.path.join('nengo', 'params_iter_{}'.format(iteration))

    model = create_model()
    # run ann
    ann_result = run_ann(model=model,
                         train=(x_train_current, y_train_current),
                         valid=(x_valid_current, y_valid_current),
                         test=(x_test, y_test),
                         params_save_path=params_path,
                         iteration=iteration
                         )
    ann.append(ann_result)

    # run spiking network for each combination of timesteps, spike scaling and synapse and append it to the snn_results
    snn_results = []
    for variant in snn_hyper_params:
        snn_acc = run_snn(model=model,
                          test=(x_test, y_test),
                          params_load_path=params_path,
                          timesteps=variant[0],
                          scale_firing_rates=variant[1],
                          synapse=variant[2],
                          iteration=iteration
                          )
        K.clear_session() # clear session not to cause a memory leak
        snn_results.append(snn_acc)

    del model # del model not to cause a memory leak
    iteration += 1
    # append list of accuracies for each parameter variant to the snn list
    snn.append(snn_results)

features: [[[ -0.77055806  -0.76216829  -0.75329262 ...   0.9730792    1.04828787
     1.00901747]]

 [[  4.72857332   4.79228735   4.86664867 ... -15.98517323 -16.30454826
   -16.64167595]]

 [[ -2.2145133   -1.49968874  -0.77021754 ...  -6.05920124  -5.69483948
    -5.42987347]]

 ...

 [[  9.08650398   9.06958485   9.0740099  ...  18.27712059  17.41079521
    16.49032021]]

 [[  3.50287008   2.95584893   2.36093497 ...  20.93103218  21.30741882
    21.62874603]]

 [[-14.8089571  -14.57761955 -14.33197689 ...  32.55890274  32.86030197
    33.12610626]]] (8036, 1, 3600)
labels: [[[1 0]]

 [[1 0]]

 [[1 0]]

 ...

 [[0 1]]

 [[0 1]]

 [[0 1]]] (8036, 1, 2)
train_x: [[[ -0.5420959   -1.21452522  -1.89456093 ...  -1.16598654  -1.73066759
    -2.2462945 ]]

 [[ -9.35857487  -9.43084145  -9.40663052 ...  -1.80052102  -0.70815951
     0.17630903]]

 [[  4.39046383   3.40499306   2.36262703 ...  38.03783417  38.5178566
    38.89231491]]

 ...

 [[-19.15268135 -19.42535782 -19.74151993 ... -1

C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\converter.py:325: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\converter.py:325: UserWarning: average_pooling2d.padding has value same != valid, which is not supported. Falling back to TensorNode.
  warnings.warn(
C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\converter.py:325: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True or layer.trainable=False. Falling back to TensorNode.
  warnings.warn(
C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\converter.py:589: UserWarning: Activation type <function softmax_v2 at 0x000002B10439BEE0> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(


Construction finished in 0:00:00                                               
Epoch 1/30
|             Constructing graph: build stage (0%)             | ETA:  --:--:--

C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\simulator.py:1931: UserWarning: Number of elements in input data (1507) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(
C:\Users\itznu\anaconda3\envs\data-science\lib\site-packages\nengo_dl\simulator.py:1931: UserWarning: Number of elements in input data (4520) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(


282/282 [==============================] - 3s 10ms/step - loss: 0.9291 - probe_loss: 0.9291 - probe_accuracy: 0.5534 - val_loss: 0.7155 - val_probe_loss: 0.7155 - val_probe_accuracy: 0.5878
Epoch 2/30
282/282 [==============================] - 2s 8ms/step - loss: 0.7221 - probe_loss: 0.7221 - probe_accuracy: 0.5904 - val_loss: 0.6725 - val_probe_loss: 0.6725 - val_probe_accuracy: 0.6090
Epoch 3/30
282/282 [==============================] - 2s 8ms/step - loss: 0.6765 - probe_loss: 0.6765 - probe_accuracy: 0.6086 - val_loss: 0.6661 - val_probe_loss: 0.6661 - val_probe_accuracy: 0.6037
Epoch 4/30
282/282 [==============================] - 2s 8ms/step - loss: 0.6600 - probe_loss: 0.6600 - probe_accuracy: 0.6133 - val_loss: 0.6592 - val_probe_loss: 0.6592 - val_probe_accuracy: 0.6263
Epoch 5/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6548 - probe_loss: 0.6548 - probe_accuracy: 0.6234 - val_loss: 0.6568 - val_probe_loss: 0.6568 - val_probe_accuracy: 0.6323
Epoch 6/30

...

Epoch 7/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6481 - probe_loss: 0.6481 - probe_accuracy: 0.6334 - val_loss: 0.6549 - val_probe_loss: 0.6549 - val_probe_accuracy: 0.6257
Epoch 8/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6464 - probe_loss: 0.6464 - probe_accuracy: 0.6243 - val_loss: 0.6483 - val_probe_loss: 0.6483 - val_probe_accuracy: 0.6456
Epoch 9/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6431 - probe_loss: 0.6431 - probe_accuracy: 0.6343 - val_loss: 0.6485 - val_probe_loss: 0.6485 - val_probe_accuracy: 0.6277
Epoch 10/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6385 - probe_loss: 0.6385 - probe_accuracy: 0.6412 - val_loss: 0.6552 - val_probe_loss: 0.6552 - val_probe_accuracy: 0.6383
Epoch 11/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6388 - probe_loss: 0.6388 - probe_accuracy: 0.6432 - val_loss: 0.6536 - val_probe_loss: 0.6536 - val_probe_accuracy: 0.635

Epoch 13/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6328 - probe_loss: 0.6328 - probe_accuracy: 0.6483 - val_loss: 0.6522 - val_probe_loss: 0.6522 - val_probe_accuracy: 0.6343
Epoch 14/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6345 - probe_loss: 0.6345 - probe_accuracy: 0.6487 - val_loss: 0.6619 - val_probe_loss: 0.6619 - val_probe_accuracy: 0.6031
Epoch 15/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6337 - probe_loss: 0.6337 - probe_accuracy: 0.6403 - val_loss: 0.6564 - val_probe_loss: 0.6564 - val_probe_accuracy: 0.6177
Epoch 16/30
282/282 [==============================] - 2s 8ms/step - loss: 0.6333 - probe_loss: 0.6333 - probe_accuracy: 0.6385 - val_loss: 0.6591 - val_probe_loss: 0.6591 - val_probe_accuracy: 0.6243
Epoch 00016: early stopping
125/125 [==============================] - 0s 3ms/step - loss: 0.6436 - probe_loss: 0.6436 - probe_accuracy: 0.6370
22. ann accuracy: 63.700002%
22. snn [timesteps=30, 

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
282/282 [==============================] - 3s 10ms/step - loss: 0.8995 - probe_loss: 0.8995 - probe_accuracy: 0.5414 - val_loss: 0.6786 - val_probe_loss: 0.6786 - val_probe_accuracy: 0.5977
Epoch 2/30
282/282 [==============================] - 2s 8ms/step - loss: 0.7157 - probe_loss: 0.7157 - probe_accuracy: 0.5893 - val_loss: 0.6724 - val_probe_loss: 0.6724 - val_probe_accuracy: 0.6090
Epoch 3/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6711 - probe_loss: 0.6711 - probe_accuracy: 0.6082 - val_loss: 0.6543 - val_probe_loss: 0.6543 - val_probe_accuracy: 0.6110
Epoch 4/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6612 - probe_loss: 0.6612 - probe_accuracy: 0.6190 - val_loss: 0.6566 - val_probe_l

Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
282/282 [==============================] - 3s 9ms/step - loss: 0.9244 - probe_loss: 0.9244 - probe_accuracy: 0.5530 - val_loss: 0.6837 - val_probe_loss: 0.6837 - val_probe_accuracy: 0.6070
Epoch 2/30
282/282 [==============================] - 2s 7ms/step - loss: 0.7284 - probe_loss: 0.7284 - probe_accuracy: 0.5827 - val_loss: 0.6687 - val_probe_loss: 0.6687 - val_probe_accuracy: 0.6077
Epoch 3/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6823 - probe_loss: 0.6823 - probe_accuracy: 0.6024 - val_loss: 0.6489 - val_probe_loss: 0.6489 - val_probe_accuracy: 0.6190
Epoch 4/30
282/282 [==============================] - 2s 7ms/step - loss: 0.6649 - probe_loss: 0.6649 - probe_accuracy: 0.6172 - val_loss: 0.6455 - val_probe_loss: 0.6455 - val_probe_accuracy: 0.6350
Epoch 5/30
282/282 [====================

282/282 [==============================] - 3s 11ms/step - loss: 0.9463 - probe_loss: 0.9463 - probe_accuracy: 0.5552 - val_loss: 0.6930 - val_probe_loss: 0.6930 - val_probe_accuracy: 0.5911
Epoch 2/30
282/282 [==============================] - 2s 8ms/step - loss: 0.7367 - probe_loss: 0.7367 - probe_accuracy: 0.5742 - val_loss: 0.6703 - val_probe_loss: 0.6703 - val_probe_accuracy: 0.6137
Epoch 3/30
282/282 [==============================] - 2s 8ms/step - loss: 0.6734 - probe_loss: 0.6734 - probe_accuracy: 0.6117 - val_loss: 0.6546 - val_probe_loss: 0.6546 - val_probe_accuracy: 0.6310
Epoch 4/30
282/282 [==============================] - 2s 8ms/step - loss: 0.6577 - probe_loss: 0.6577 - probe_accuracy: 0.6210 - val_loss: 0.6633 - val_probe_loss: 0.6633 - val_probe_accuracy: 0.6037
Epoch 5/30
282/282 [==============================] - 3s 9ms/step - loss: 0.6523 - probe_loss: 0.6523 - probe_accuracy: 0.6285 - val_loss: 0.6717 - val_probe_loss: 0.6717 - val_probe_accuracy: 0.6190
Epoch 6/30

In [9]:
def visualize(ann, snn, snn_hyper_params, xlsx_output, graph_output, graph_format='svg'):
    data = {
        'iterations': [x for x in range(1, len(ann) + 1)],
        'ann_accuracy': ann,
    }

    variants = []
    for variant in snn_hyper_params:
        if variant[2] is None:
            variants.append('snn: timesteps={}, scaling={:3f}, synapse=None'
                            .format(variant[0], variant[1]))
        else:
            variants.append('snn: timesteps={}, scaling={:3f}, synapse={:3f}'
                        .format(variant[0], variant[1], variant[2]))

    snn_accs_by_variant = [[] for _ in range(len(variants))]
    for snn_acc_list in snn:
        for i in range(len(snn_acc_list)):
            snn_accs_by_variant[i].append(snn_acc_list[i])

    for i in range(len(variants)):
        data[variants[i]] = snn_accs_by_variant[i]

    df = pd.DataFrame(data=data, index=list(range(1, len(ann) + 1)))
    print(df)
    df.to_excel(xlsx_output)

    ann_avg = statistics.mean(ann)
    print('Average accuracy of ANN: {:5f}%'.format(ann_avg*100))

    for i in range(len(variants)):
        snn_average = statistics.mean(snn_accs_by_variant[i])
        print('Average accuracy of {}: {:5f}%'.format(variants[i], snn_average*100))

    df.plot(x='Iterations', y=variants, kind='bar', title='Accuracy of ANN and respective variant of SNN per iteration')
    plt.yticks(np.arange(0, max(max(snn), max(ann)), 0.05))
    plt.xticks(rotation=0)
    plt.ylabel('Accuracy')
    plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    plt.savefig(graph_output, format=graph_format, bbox_inches='tight')
    plt.show()

In [10]:
visualize(ann=ann,
          snn=snn,
          snn_hyper_params=snn_hyper_params,
          xlsx_output=os.path.join('output', 'values.xlsx'),
          graph_output=os.path.join('output', 'graph.svg')
)



ValueError: could not broadcast input array from shape (30) into shape (0)